In [1]:
from preprocessing import Preproc
import pandas as pd
import numpy as np

In [2]:
path = 'https://raw.githubusercontent.com/EraylsonGaldino/dataset_time_series/master/airline.txt'
df_serie = pd.read_csv(path, header=None)

In [3]:
pre = Preproc()

In [4]:
serie_np = df_serie.to_numpy().reshape(-1, )
dict_samples = pre.split_serie(serie_np, perc_train = 0.6)
train, test = dict_samples['train'], dict_samples['test']



In [5]:
_ = pre.normalise_interval(train)

In [6]:
train_norm = pre.normalise_interval(train)
test_norm = pre.normalise_interval(test)

In [7]:
idx_lags = pre.select_lag_acf( train_norm, 20)

LAGS [0, 1, 2, 3, 4, 5]


In [8]:
X_train, y_train = pre.create_windows(train_norm, np.max(idx_lags)+1)


In [9]:
qtd_lags = np.max(idx_lags)
test_with_previous = np.vstack([train_norm[-qtd_lags-1:], test_norm])
X_test, y_test = pre.create_windows(test_with_previous, np.max(idx_lags)+1)

In [10]:
X_train = X_train[:, idx_lags]
X_test = X_test[:, idx_lags]

In [11]:
from ensemble import Ensemble

In [21]:
ens = Ensemble(100, 'SVR', approach_generate='bagging')

In [ ]:
ens.fit(X_train, y_train)

In [14]:
ens.pool

{'models': [SVR(C=10000, epsilon=0.01, gamma=0.1),
  SVR(C=1, epsilon=0.001, gamma=20)],
 'indices': [[51,
   14,
   71,
   60,
   20,
   74,
   74,
   23,
   2,
   21,
   52,
   1,
   29,
   37,
   1,
   63,
   59,
   20,
   32,
   75,
   57,
   21,
   48,
   58,
   41,
   59,
   79,
   14,
   61,
   61,
   46,
   61,
   50,
   54,
   63,
   2,
   50,
   6,
   20,
   72,
   38,
   17,
   3,
   59,
   13,
   8,
   52,
   1,
   59,
   70,
   43,
   7,
   46,
   34,
   77,
   35,
   49,
   3,
   1,
   5,
   53,
   3,
   53,
   62,
   17,
   43,
   33,
   73,
   61,
   13,
   47,
   14,
   71,
   77,
   61,
   39,
   79,
   52,
   23,
   25],
  [4,
   36,
   71,
   30,
   8,
   50,
   28,
   77,
   39,
   40,
   10,
   22,
   0,
   45,
   20,
   35,
   53,
   56,
   0,
   62,
   53,
   54,
   39,
   14,
   20,
   46,
   72,
   52,
   8,
   73,
   51,
   56,
   25,
   40,
   34,
   62,
   24,
   74,
   37,
   1,
   6,
   33,
   16,
   42,
   58,
   50,
   53,
   23,
   24,
   70,
   51,
  

In [15]:
ens.save_models('airline', 'svr')

In [17]:
from joblib import load

In [18]:
model = load('/home/eraylson/union/airline/svr/model_0.joblib')

In [20]:
model.predict(X_test[0].reshape(1, -1))

array([0.74227372])

In [ ]:
X_test[0].shape

In [ ]:
len(X_test[0].shape)

In [ ]:
X_test[0].reshape(1, -1).shape

In [ ]:
X_test.shape

In [ ]:
ens.predict(X_test[0])

In [ ]:
y_pred = ens.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt 



In [ ]:
plt.plot(y_pred, label = 'pred')
plt.plot(y_test, label = 'target')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
MSE(y_pred, y_test) #0.02836